## Q.31. アフィン変換(スキュー)

(1)アフィン変換を用いて、出力(1)のようなX-sharing(dx = 30)画像を作成せよ。

(2)アフィン変換を用いて、出力2のようなY-sharing(dy = 30)画像を作成せよ。

(3)アフィン変換を用いて、出力3のような幾何変換した(dx = 30, dy = 30)画像を作成せよ。

このような画像はスキュー画像と呼ばれ、画像を斜め方向に伸ばした画像である。

出力(1)の場合、x方向にdxだけ引き伸ばした画像はX-sharingと呼ばれる。

出力(2)の場合、y方向にdyだけ引き伸ばした画像はY-sharingと呼ばれる。

それぞれ次式のアフィン変換で実現できる。
ただし、元画像のサイズがh x wとする。

```bash
(1) X-sharing                  (2) Y-sharing
   a = dx / h                     a = dy / w

  x'       1 a tx    x           x'       1 0 tx    x
[ y' ] = [ 0 1 ty ][ y ]       [ y' ] = [ a 1 ty ][ y ]
  1        0 0  1    1           1        0 0  1    1
```

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Affine
def affine(img, dx=30, dy=30):
  H, W, C = img.shape

  # Affine hyper parameters
  a = 1.
  b = dx / H
  c = dy / W
  d = 1.
  tx = 0.
  ty = 0.

  # prepare temporary
  _img = np.zeros((H+2, W+2, C), dtype=np.float32)

  # insert image to center of temporary
  _img[1:H+1, 1:W+1] = img

  # prepare affine image temporary
  H_new = np.ceil(dy + H).astype(np.int16)
  W_new = np.ceil(dx + W).astype(np.int16)
  out = np.zeros((H_new, W_new, C), dtype=np.float32)

  # preprare assigned index
  x_new = np.tile(np.arange(W_new), (H_new, 1))
  y_new = np.arange(H_new).repeat(W_new).reshape(H_new, -1)

  # prepare inverse matrix for affine
  adbc = a * d - b * c
  x = np.round((d * x_new  - b * y_new) / adbc).astype(np.int16) - tx + 1
  y = np.round((-c * x_new + a * y_new) / adbc).astype(np.int16) - ty + 1

  x = np.minimum(np.maximum(x, 0), W+1).astype(np.int16)
  y = np.minimum(np.maximum(y, 0), H+1).astype(np.int16)

  # assign value from original to affine image
  out[y_new, x_new] = _img[y, x]
  out = out.astype(np.uint8)

  return out

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# Affine
out = affine(img, dx=30, dy=30)
cv2.imwrite("training_IMG/training_31.png", out)

True